In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://github.com/TaewoongKong/code_sharing/blob/master/cau_shops_kakao_edit.csv?raw=True')
#df = pd.read_csv('cau_shops_kakao_edit.csv')

In [3]:
# 필요 칼럼만 이용

df_recom = df[['카카오지도_상호명',
               '상권업종중분류명',
               '상권업종소분류명',
               '카카오지도_카테고리',
               '경도',
               '위도',
               '카카오지도_별점_점수',
               '카카오지도_별점_평가수',
               '카카오지도_블로그_리뷰',
               '카카오지도_블로그_리뷰수']]


In [4]:
# 칼럼명 단순화

df_recom.columns = ['name',  # 상호명
                    'cate_1',  
                    'cate_2', 
                    'cate_3',
                    'lon',
                    'lat',
                    'star_point',  # 별점 점수
                    'star_qty',  # 별점 평가 횟수
                    'review_txt',  # 리뷰 텍스트
                    'review_qty'  #  리뷰 횟수 - 유명할수록 많은데 부정적인 평가도 있을 수 있다
                   ]

In [5]:
# 별점 평가 개수가 0개면 별점을 -1 로 Null 처리

df_recom.loc[df_recom['star_qty'] == 0, 'star_point'] = -1



# 각종 카테고리를 통합
df_recom['cate_mix'] = df_recom['cate_1'] + df_recom['cate_2'] + df_recom['cate_3']
df_recom['cate_mix'] = df_recom['cate_mix'].str.replace(">", " ").str.replace("/", " ")

/Users/taewoongkong/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/taewoongkong/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/taewoongkong/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# 리뷰의 유사도 학습


count_vect_review = CountVectorizer(min_df=2, ngram_range=(1,2))
place_review = count_vect_review.fit_transform(df_recom['review_txt']) 
place_simi_review = cosine_similarity(place_review, place_review)
place_simi_review_sorted_ind = place_simi_review.argsort()[:, ::-1]


# 카테고리의 유사도 학습

count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df_recom['cate_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1] 


# 카테고리와 리뷰의 중요성을 짬뽕시키는 공식
place_simi_co = (place_simi_review*0.2  # 리뷰 유사도는 0.1만큼 반영
                 + place_simi_cate*0.05 
                 + np.repeat([df_recom['star_qty'].values], len(df_recom['star_qty']) , axis=0) * 0.001  # 별점평가 갯수
                 + np.repeat([df_recom['star_point'].values], len(df_recom['star_point']) , axis=0) * 0.001 # 별점 평가 반영
                )


# 필요한 작업
place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 



In [12]:
# 위경도 구하는 공식 

def haversine_distance(lat1, lon1, lat2, lon2):
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 2)



# 추천엔진 돌리는 최종 함수

def find_simi_place(df, sorted_ind, place_name, top_n=10):
    
    
    
    place_title = df[df['name'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    result_df = df.iloc[similar_indexes]
    
    start_lat, start_lon = result_df['lat'].tolist()[0], result_df['lon'].tolist()[0]
    
    distances_km = []
    for row in result_df.itertuples(index=False):
        distances_km.append(haversine_distance(start_lat, start_lon, row.lat, row.lon))
    
    
    result_df['distance'] = distances_km
    
    

    return result_df.sort_values(by='distance')  # 위경도 거리를 가까운 순으로 점수 올려서 순서를 바꿔줘도 되고... (sorting 이용)

In [13]:
find_simi_place(df_recom,place_simi_co_sorted_ind, "BHC치킨 중앙대점", 7)

/Users/taewoongkong/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,name,cate_1,cate_2,cate_3,lon,lat,star_point,star_qty,review_txt,review_qty,cate_mix,distance
25,BHC치킨 중앙대점,닭/오리요리,후라이드/양념치킨,음식점 > 치킨 > BHC치킨,126.958410,37.507019,4.0,2,안녕하세요~! 부동산 발자국입니다. 치킨이 땡기는 날이라 치킨을 먹었습니다. 흑서동...,5,닭 오리요리후라이드 양념치킨음식점 치킨 BHC치킨,0.00
89,치폴레옹,닭/오리요리,후라이드/양념치킨,음식점 > 치킨,126.960671,37.507967,4.2,10,성인이 된 이후 이사온 흑석동 처음엔 쭉 자라온 동네를 떠나오는게 너무 싫고 동네친...,22,닭 오리요리후라이드 양념치킨음식점 치킨,0.23
110,우뇽뚝배기스파게티,양식,스파게티전문점,음식점 > 양식 > 이탈리안,126.961633,37.506300,3.7,69,중앙대병원에 서류를 좀 발급받으러 갔다.. 아따 비싸네...... 근데 물어보니 무...,40,양식스파게티전문점음식점 양식 이탈리안,0.30
175,호치킨 중앙대점,닭/오리요리,후라이드/양념치킨,음식점 > 치킨 > 호치킨,126.962672,37.508012,3.1,10,퇴근길에 치킨이 땡겨서 치킨을 포장해왔는데요~ 오늘은 외근으로 중앙대병원 근처 왔다...,29,닭 오리요리후라이드 양념치킨음식점 치킨 호치킨,0.39
9,교촌치킨 상도터널점,닭/오리요리,후라이드/양념치킨,음식점 > 치킨 > 교촌치킨,126.951280,37.504911,1.8,23,교촌치킨 동작구 상도터널점 메뉴 및 영업시간 ＊누구나 즐기는 간식거리 치킨!! 그중...,2,닭 오리요리후라이드 양념치킨음식점 치킨 교촌치킨,0.67
172,사리원,분식,국수/만두/칼국수,음식점 > 한식 > 냉면,126.950615,37.499147,3.1,111,안녕하세요 공공일이에요! 저는 티엠아이지만. 저희 할아버지께서 이북에서 오셔서 이북...,42,분식국수 만두 칼국수음식점 한식 냉면,1.11
293,BHC치킨 숭실대점,닭/오리요리,후라이드/양념치킨,음식점 > 치킨 > BHC치킨,126.957436,37.494928,3.1,11,종강까지의 일상 드디어 내가 종강을 하긴 하는구나 어찌됐든 이번 학기도 잘 버텼다 ...,10,닭 오리요리후라이드 양념치킨음식점 치킨 BHC치킨,1.35


In [14]:

# 네이버 블로그 리뷰와 별점 평가가 필요한 항목들을 아래 코드를 돌리면 1과 Null로 구분되게 짜두었습니다
# 1로 표기된 상호들만 크롤링 해주시면 됩니다
# 데이터의 수는 21개 뿐입니다 (인간지능 해도 될듯...?)

df_recom['naver_blog_need'] = np.nan


df_recom.loc[(df_recom['review_txt'] == 'empty')
             & (df_recom['star_qty'] == 0)].index.tolist()

df_recom.loc[df_recom.index.isin(df_recom.loc[(df_recom['review_txt'] == 'empty') & (df_recom['star_qty'] == 0)].index.tolist()), 'naver_blog_need'] = 1




/Users/taewoongkong/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/taewoongkong/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
